In [2]:
import nltk,re,time,umap,collections,os,string,sys,mlflow
sys.path.append(os.path.abspath(".."))
from modules import utils,modeling

from sentence_transformers import SentenceTransformer
import plotly.express as px
import pandas as pd
import pickle

from sklearn.metrics import completeness_score,homogeneity_score, v_measure_score,normalized_mutual_info_score,adjusted_mutual_info_score,silhouette_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

from sentence_transformers import models

from hdbscan import HDBSCAN
from sentence_transformers import models
from sklearn.preprocessing import normalize
from transformers import AutoModel, AutoTokenizer
from torch import nn
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
from sklearn.preprocessing import StandardScaler

remote_server="http://ec2-18-193-70-141.eu-central-1.compute.amazonaws.com"
client = MlflowClient(remote_server)
mlflow.set_tracking_uri(remote_server)


%matplotlib inline
%load_ext autoreload
%autoreload 2

Number of stopwords 607


# UMAP Training 

UMAP as a mainfolf aproximation method performs best on a previously un-seen data when it "pre trained". This notebook fits the UMAP model with the generated emebddings. 


### Load data and the model 

In [18]:
#model = SentenceTransformer('bert-base-wikipedia-sections-mean-tokens')
TOP_N = 1000
TRAIN_N = 100000
MODEL_NAME="bert-german-dbmdz-uncased-sentence-stsb"
start_time = time.time()


df = utils.load_text_data().head(TRAIN_N).dropna(subset=['text'])
df = df[df.text.apply(lambda x: isinstance(x, str))]

print("--- Data Loaded: "+str(df.shape)+" ---")
# load BERT model from Hugging Face
word_embedding_model = models.Transformer(
    'T-Systems-onsite/{}'.format(MODEL_NAME))

print("--- Model & data loaded in %s seconds ---" % (time.time() - start_time))
df.head()

--- Data Loaded: (100000, 8) ---
--- Model & data loaded in 8.736830949783325 seconds ---


,kicker,headline,text,seo_title,created_at,updated_at,site,entities_analyzed
article_uid,,,,,,,,
8c7b60bbbbab27058f93ba3114462b0b9b82f79c3cf3e95e52cf8013742b1f89,Tor nach 3 Sekunden!,Blitz-Comeback von Reus,Wenn du bei deinem Comeback nach 223 Tagen nur...,DFB-Pokal: Duisburg - BVB 0:5! 3 Sekunden nach...,2020-09-14 22:39:16,2020-09-14 22:39:16,www.bild.de,0
66ef992ac10f512ce621314bcffcbc542549ba1f52c8b11696e33d8730998aba,Sie stoppten das Schalke-Spiel,Türkgücü-Bosse erklärenihre Pokal-Klage,Es war DER Aufreger der 1. DFB-Pokal-Runde. T...,DFB-Pokal: Sie stoppten das Schalke-Spiel – Tü...,2020-09-14 22:34:15,2020-09-14 22:35:05,www.bild.de,1
bafa8f876a14f50c6010a8ea2ff4de2265e508af13c5007ddb363a4c4b155f4e,Messer-Angriff von Stolberg,Islamist stachschon einmal zu!,Stolberg/Aachen – Der Messerangriff von Stolbe...,Messer-Angriff von Stolberg - Islamist stach...,2020-09-14 22:34:15,2020-09-14 22:35:05,www.bild.de,1
e353f0b8121a6f0addf3ea3ca5eead419c904d283284fcffcf913ff6f2f39b1c,Hertha verschenkt Machtzentrale,Die neuen Hierarchien der 18 Klubs,Die neue Serie „Die neuen Hierarchien der 18 K...,1. Bundesliga: Die neuen Hierarchien der 18 Kl...,2020-09-14 22:34:15,2020-09-14 22:35:05,www.bild.de,1
f168dc072b52b38187937175639830ef066209100ffaeea780cb03cd943706d3,Verletzte und neue Offensive,Gisdol schon vorm Start unter Druck!,Das hatte sich Markus Gisdol (50) bei seiner V...,1. FC Köln: Markus Gisdol schon vor Liga-Start...,2020-09-14 22:33:16,2020-09-14 22:34:07,www.bild.de,1


In [19]:
pdf = utils.preprocess_articles_for_bert(df)
df['mix'] = df['headline']+". "+df['kicker'] + ". "+" " + df["seo_title"].apply(utils.remove_seo_title_marker)
df['mix'] = df['mix'].astype(str)
#pdf_mix = utils.preprocess_text(df,col="mix")
#pdf_mix[0][:250]
for nr in  range(50,55):
    print(df.text.values[nr][:100],"\n",pdf[nr][:50])
    print()


Die neue Serie „Die neuen Hierarchien der 18 Klubs“ – heute mit Gladbach, Union Berlin, Wolfsburg un 
 Die neue Serie Die neuen Hierarchien der 18 Klubs“

Das ging fix! Während seine Kollegen am Montag in Dresden im DFB-Pokal spielten, packte Julian Polle 
 Das ging fix! Während seine Kollegen am Montag in 

Stolberg/Aachen – Der Verdächtige (21), der am Wochenende in Stolberg (NRW) auf einen Mann eingestoc 
 Stolberg Aachen Der Verdächtige, der am Wochenende

Niedersachsen – Es ist einer der wenigen positiven Corona-Effekte: Die Zahl der Straftaten sinkt! Be 
 Niedersachsen Es ist einer der wenigen positiven C

Celle – Großeinsatz der Polizei und von Rettungskräften in der Innenstadt von Celle! Ein Juwelier ha 
 Celle Großeinsatz der Polizei und von Rettungskräf



In [30]:
start_time = time.time()

path_to_embeddings ='../models/{}/Top-{}k_articles_embeddings.npy'.format(MODEL_NAME,str(TRAIN_N)[:3])
path_to_embeddings_with_ids = '../models/{}/Top-{}k_articles_embddings_ids.npy'.format(MODEL_NAME,str(TRAIN_N)[:2])

if not os.path.exists(path_to_embeddings):
    
    embeddings = modeling.get_sentence_embeddings(pdf,word_embedding_model)
    
    with open(path_to_embeddings, 'wb') as f:
        np.save(f, embeddings)

    with open(path_to_embeddings_with_ids, 'wb') as f:    
        np.save(f, list(zip( df.index.to_numpy(),embeddings)))
else:
    embeddings= np.load(path_to_embeddings)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.3611130714416504 seconds ---


In [ ]:
viz_n_neighbors= 15
start_time = time.time()

fitted_umap_viz = umap.UMAP(n_neighbors=viz_n_neighbors, n_components=2, random_state=0, metric='cosine').fit(embeddings)

pkl_filename = "umap_viz_{}_{}-neighbors.pkl".format(str(TRAIN_N)[:2],viz_n_neighbors)
with open("../models/{}/{}".format(MODEL_NAME,pkl_filename), 'wb') as file:
    pickle.dump(fitted_umap_viz, file)
print("--- %s seconds ---" % (time.time() - start_time))

In [36]:
n_neighbors = 12
n_components = 128
start_time = time.time()

fitted_umap_clustering = umap.UMAP(n_neighbors=n_neighbors, min_dist=0.05, random_state=0, metric='cosine',
                                       n_components=n_components).fit(embeddings)

pkl_filename = "umap_{}k_{}-neighbors_{}-comps.pkl".format(str(TRAIN_N)[:3],n_neighbors,n_components)
with open("../models/{}/{}".format(MODEL_NAME,pkl_filename), 'wb') as file:
    pickle.dump(fitted_umap_clustering, file)
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 417.8817672729492 seconds ---


In [27]:
#with open(path_to_embeddings_with_ids, 'r') as f:        


In [28]:
len(embeddings)

100000

True